<a href="https://colab.research.google.com/github/kitomicms/python_chess_/blob/master/python_chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# python chess
# class Game():
#   def __init__(self):
#     board = Board()
#     self.board = board
#     self.turn = 'W'

class Board():
  def __init__(self):
        files = ['A','B','C','D','E','F','G','H']
        ranks = [1,2,3,4,5,6,7,8]
        board = []
        counter = 1

        # create all the squares
        for rank in reversed(ranks):
          for file in files:
            board.append(Square(rank,file,counter))
            counter += 1

        # add coordinates
        row_counter = 1
        for count,square in enumerate(board):
          
          if (count+1) % 8 == 0:
            x = 8
            y = row_counter
            row_counter +=1
          else:
            x = (count+1) % 8
            y = row_counter
          square.coordinate = (x,y)
            

        # add pawns
        for square in board:
          if square.rank == 2:
            square.addPiece("P","W")
          elif square.rank == 7:
            square.addPiece("P","B")
        # add other pieces
        abc = ['R','N','B','Q','K','B','N','R']
        counter = 0
        for square in board:
          if square.rank == 1:
            square.addPiece(abc[counter],"W")
            counter += 1
        counter = 0
        for square in board:
          if square.rank == 8:
            square.addPiece(abc[counter],"B")
            counter += 1
        self.board = board
        
        # set initial turn
        self.turn = 'W'

  def current_turn(self):
    return self.turn
  def display_color(self):
    display_str = ''
    for count,square in enumerate(self.board):
      if (count+1) % 8 == 0:
          display_str = f"{display_str}{square.piece_color} "
          print(display_str)
          display_str = ''
      else:
        display_str = f"{display_str}{square.piece_color} "

  def display_matrix(self):
    display_str = ''
    for count,square in enumerate(self.board):
      if (count+1) % 8 == 0:
          display_str = f"{display_str}{square.coordinate} "
          print(display_str)
          display_str = ''
      else:
        display_str = f"{display_str}{square.coordinate} "

  def display_board(self):
      display_str = ''
      for count,square in enumerate(self.board):
        if (count+1) % 8 == 0:
          if square.piece_name in ['X',None]:
            display_str = f"{display_str}{square.coordinate[0]}{square.coordinate[1]} "
          else:
            display_str = f"{display_str}{square.piece_color}{square.piece_name} "
          print(display_str)
          display_str = ''
        else:
          if square.piece_name in ['X',None]:
            display_str = f"{display_str}{square.coordinate[0]}{square.coordinate[1]} "
          else:
            display_str = f"{display_str}{square.piece_color}{square.piece_name} "
      #print(display_str)
  
  def display_occupied(self):
    display_str = ''
    for count,square in enumerate(self.board):
      if (count+1) % 8 == 0:
          display_str = f"{display_str}{square.isOccupied} "
          print(display_str)
          display_str = ''
      else:
        display_str = f"{display_str}{square.isOccupied} "

  def square_info(self,rank,file):
    # what is on the square?
    
    for square in self.board:
      if square.rank == rank and square.file == file:
        return [square,square.piece_color,square.piece_name,square.coordinate]


  def makeMove(self,square1,square2):
    sq1_ = self.square_info(int(square1[1]),square1[0])[0]
    sq1_color = self.square_info(int(square1[1]),square1[0])[1]
    sq1_pieceName = self.square_info(int(square1[1]),square1[0])[2]
    sq2_ = self.square_info(int(square2[1]),square2[0])[0]
    sq2_color = self.square_info(int(square2[1]),square2[0])[1]
    sq2_pieceName = self.square_info(int(square2[1]),square2[0])[2]

    if self.turn != sq1_color:
      print("Not your turn")
      return 0

    if sq1_.isOccupied != False:
      if sq2_.isOccupied == False:
        sq1_.reset()
        sq2_.reset()
        sq2_.addPiece(sq1_pieceName,sq1_color)
        self.turn = 'W' if self.turn == 'B' else 'B'

      elif sq2_.isOccupied and sq1_color != sq2_color:
        sq1_.reset()
        sq2_.reset()
        sq2_.addPiece(sq1_pieceName,sq1_color)
        self.turn = 'W' if self.turn == 'B' else 'B'
      elif sq1_color == sq2_color:
        print("both square has same color piece")
      else:
        print("something else went wrong")
    else:
      print("Square 1 is Empty")
    # 1. MT
    # 2. Occupied,MT (King exception, path checked by opponent)
    # 3. Occupied,Occupied
    # 3.1 W,W
    # 3.2 W,B
  
  def availableMoves(self,square1):
    # according to current piece position and apply coordinate movement
    sq1_ = self.square_info(int(square1[1]),square1[0])[0]
    sq1_pieceColor = sq1_.piece_color
    sq1_x = sq1_.coordinate[0]
    sq1_y = sq1_.coordinate[1]
    
    # check if the square is occupied
    if sq1_.isOccupied:
      movesIndex = sq1_.piece.validMovesIndex
      possibleMoves = [(x+sq1_x,y+sq1_y) for (x,y) in movesIndex]
      impossibleMoves = []
      
      # find out moves that are not in the 8x8 grid
      for move in possibleMoves:
        if (move[0] <= 0) or (move[1] <= 0):
          impossibleMoves.append(move)
        elif (move[0] >= 9) or (move[1] >= 9):
          impossibleMoves.append(move)
      # remove all the impossible moves
      for move in impossibleMoves:
        possibleMoves.remove(move)
      impossibleMoves = []

      # check if dest position has own piece
      for move in possibleMoves:
        if self.squareByCoordinate(move).piece_color == sq1_pieceColor:
          impossibleMoves.append(move)
      # remove all the impossible moves
      for move in impossibleMoves:
        possibleMoves.remove(move)
      impossibleMoves = []
      # find out if other pieces is between current position and the dest position
      # Pawn
      ## 1. move forward
      ## 2. move diagonal to take opponent piece
      ## 3. en passant to take opponent piece, need to recall pieces move

      # Knight
      ## no same color piece at dest position
      
      # King
      ## no same color piece at dest position
      
      # applied to Queen/Rook/Bishop
      if sq1_.piece_name in ['Q','R','B']:  
        for (x_,y_) in possibleMoves:
          x_distance = x_ - sq1_x
          y_distance = y_ - sq1_y

          if (abs(x_distance) == 1 or abs(x_distance) == 0) and \
          (abs(y_distance) == 1 or abs(y_distance) == 0): # all the 1 square moves
            pass
          elif (abs(x_distance) > 1 and abs(y_distance) == 0):
            # say rook move sideway
            if x_distance > 1:
              for x in range(1,x_distance):
                if self.squareByCoordinate((sq1_x+x,sq1_y)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
            elif x_distance < 1:
              for x in range(1,-x_distance):
                if self.squareByCoordinate((-sq1_x+x,sq1_y)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
          elif (abs(x_distance) == 0 and abs(y_distance) > 1):
            # say rook move up and down
            if y_distance > 1:
              for y in range(1,y_distance):
                if self.squareByCoordinate((sq1_x,y+sq1_y)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
            elif y_distance < 1:
              for y in range(1,-y_distance):
                if self.squareByCoordinate((sq1_x,-y+sq1_y)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
          elif abs(x_distance) == abs(y_distance):
            if x_distance > 1 and y_distance > 1:
              for x in range(1,x_distance):
                if self.squareByCoordinate((sq1_x+x,sq1_y+x)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
            elif x_distance > 1 and y_distance < 1:
              for x in range(1,x_distance):
                if self.squareByCoordinate((sq1_x+x,sq1_y-x)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
            elif x_distance < 1 and y_distance > 1:
              for x in range(1,-x_distance):
                if self.squareByCoordinate((sq1_x-x,sq1_y+x)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
            elif x_distance < 1 and y_distance < 1:
              for x in range(1,-x_distance):
                if self.squareByCoordinate((sq1_x-x,sq1_y-x)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
            else:
              print("Weird")  
          else: # should remain diagonal move
            print("Weird move")
        # remove all the impossible moves
        for move in impossibleMoves:
          possibleMoves.remove(move)
        impossibleMoves = []
      return possibleMoves # all the legit move based on piece's rules
    else:
      print("This square is not occupied by any pieces")

    # remove the invalid move due to blockage
    
  def squareByCoordinate(self,coordinate):
    # x files , y ranks
    files = ['A','B','C','D','E','F','G','H']
    ranks = [1,2,3,4,5,6,7,8]
    return self.square_info(ranks[-(coordinate[1])],files[coordinate[0]-1])[0]



  def validMoves(square1,possibleMoves):
    # need to check if being checked after move, king's path

    # king move
    ## still being checked?

    # other move
    ## allow to be checked?
    pass
  def isChecked(turn):
    # checked or checkmated
    # find King position
    # check all the path to see if certain piece exists
    pass
  def isCheckmated():
    pass
  def isStalemate():
    pass
  def castling():
    pass
  def enpassant():
    pass
    # need to check if oppenent pawn made 2 step move at first and never move after that

# row = rank, col = file
class Square():
  def __init__(self, rank,file,coordinate,piece=None):
        self.rank = rank
        self.file = file
        self.isOccupied = False
        self.piece = None
        self.piece_name = 'X'
        self.piece_color = 'X'
        self.coordinate = coordinate

  def addPiece(self,name,color):
    if name == 'P':
      self.piece = Pawn(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'K':
      self.piece = King(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'Q':
      self.piece = Queen(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'R':
      self.piece = Rook(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'N':
      self.piece = Knight(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'B':
      self.piece = Bishop(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    else:
      print("invalid piece name entered")
      

  def reset(self):
    self.isOccupied = False
    self.piece = None
    self.piece_name = 'X'
    self.piece_color = 'X'

class Piece():
  def __init__(self,name,color):
    self.name = name
    self.color = color
    

class King(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = []
    self.validMovesIndex.append((1,0))
    self.validMovesIndex.append((-1,0))
    self.validMovesIndex.append((0,1))
    self.validMovesIndex.append((0,-1))
    self.validMovesIndex.append((1,1))
    self.validMovesIndex.append((1,-1))
    self.validMovesIndex.append((-1,1))
    self.validMovesIndex.append((-1,-1))
class Queen(Piece):
  # moves rule
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = []
    for x in [1,2,3,4,5,6,7]:
      self.validMovesIndex.append((x,0))
      self.validMovesIndex.append((-x,0))
      self.validMovesIndex.append((0,x))
      self.validMovesIndex.append((0,-x))
      self.validMovesIndex.append((x,x))
      self.validMovesIndex.append((x,-x))
      self.validMovesIndex.append((-x,x))
      self.validMovesIndex.append((-x,-x))
class Rook(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = []
    for x in [1,2,3,4,5,6,7]:
      self.validMovesIndex.append((x,0))
      self.validMovesIndex.append((-x,0))
      self.validMovesIndex.append((0,x))
      self.validMovesIndex.append((0,-x))
      
class Pawn(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    # depends on which color
    self.validMovesIndex = []
    if self.color == 'W':
      self.validMovesIndex.append((0,-1))
      # what if it is the first move
    else:
      self.validMovesIndex.append((0,1))
    self.movesHistory = []

class Knight(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = []
    self.validMovesIndex.append((1,2))
    self.validMovesIndex.append((-1,2)) 
    self.validMovesIndex.append((1,-2)) 
    self.validMovesIndex.append((-1,-2)) 
    self.validMovesIndex.append((2,1)) 
    self.validMovesIndex.append((-2,1)) 
    self.validMovesIndex.append((2,-1))  
    self.validMovesIndex.append((-2,-1))  

class Bishop(Piece):
  
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = []
    for x in [1,2,3,4,5,6,7]:
      self.validMovesIndex.append((x,x))
      self.validMovesIndex.append((x,-x))
      self.validMovesIndex.append((-x,x))
      self.validMovesIndex.append((-x,-x))

def test_move(case,answer):
  case = sorted(case)
  answer = sorted(answer)
  
  if case == answer:
    
    print("Test passed")
  else:
    print(case,"is passed through")
    print(answer,"is expected")
    print("Test failed")


In [ ]:
board = Board()
board.current_turn()
board.makeMove("H1","D4")
board.makeMove("H7","H6")
board.makeMove("D2","D3")
board.makeMove("B8","E4")
board.makeMove("H2","H3")
board.makeMove("E7","E5")
board.makeMove("B2","B3")
board.display_board()

# Test black rook
test_move(board.availableMoves("H8"),[(8,2)])

# Test white knight
test_move(board.availableMoves("G1"),[(6,6)])
# Test Black knight
test_move(board.availableMoves("E4"),[(4,3),(6,3),(3,4),(3,6),(4,7),(6,7),(7,4),(7,6)])

# Test white pawn
test_move(board.availableMoves("H3"),[(8,5)])

# Test white pawn
test_move(board.availableMoves("A8"),[(2,1)])

# Test white pawn
test_move(board.availableMoves("D3"),[])

# Test white bishop
test_move(board.availableMoves("C1"),[(1,6),(2,7),(4,7),(5,6),(6,5),(7,4),(8,3)])

# Test white bishop 2
test_move(board.availableMoves("F1"),[])

# Test white queen
test_move(board.availableMoves("D1"),[(4,7)])

# Test black queen
test_move(board.availableMoves("D8"),[(5,2),(6,3),(7,4),(8,5)])

# Test black bishop
test_move(board.availableMoves("F8"),[(5,2),(4,3),(3,4),(2,5),(1,6)])

# Test black bishop 2
test_move(board.availableMoves("C8"),[])

BR 21 BB BQ BK BB BN BR 
BP BP BP BP 52 BP BP 82 
13 23 33 43 53 63 73 BP 
14 24 34 44 BP 64 74 84 
15 25 35 WR BN 65 75 85 
16 WP 36 WP 56 66 76 WP 
WP 27 WP 47 WP WP WP 87 
WR WN WB WQ WK WB WN 88 
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed


In [ ]:
board.display_matrix()
board.display_color()
board.display_occupied()

(1, 1) (2, 1) (3, 1) (4, 1) (5, 1) (6, 1) (7, 1) (8, 1) 
(1, 2) (2, 2) (3, 2) (4, 2) (5, 2) (6, 2) (7, 2) (8, 2) 
(1, 3) (2, 3) (3, 3) (4, 3) (5, 3) (6, 3) (7, 3) (8, 3) 
(1, 4) (2, 4) (3, 4) (4, 4) (5, 4) (6, 4) (7, 4) (8, 4) 
(1, 5) (2, 5) (3, 5) (4, 5) (5, 5) (6, 5) (7, 5) (8, 5) 
(1, 6) (2, 6) (3, 6) (4, 6) (5, 6) (6, 6) (7, 6) (8, 6) 
(1, 7) (2, 7) (3, 7) (4, 7) (5, 7) (6, 7) (7, 7) (8, 7) 
(1, 8) (2, 8) (3, 8) (4, 8) (5, 8) (6, 8) (7, 8) (8, 8) 
B X B B B B B B 
B B B B X B B X 
X X X X X X X B 
X X X X B X X X 
X X X W B X X X 
X W X W X X X W 
W X W X W W W X 
W W W W W W W X 
True False True True True True True True 
True True True True False True True False 
False False False False False False False True 
False False False False True False False False 
False False False True True False False False 
False True False True False False False True 
True False True False True True True False 
True True True True True True True False 


In [ ]:
for x in range(1,2):
  print(x)

1
